<!--
This file is part of cjdk.
Copyright 2022, Board of Regents of the University of Wisconsin System
SPDX-License-Identifier: MIT
--->

# Building and running a GraalVM native image

This example shows how to compile Java source code into a native image
(executable that does not require the JVM) using GraalVM.

Note that an appropriate C++ toolchain must be installed for this to work; see
the GraalVM documentation for details.

In [1]:
import cjdk
import subprocess

In [2]:
java_source = """
public class Hello {
    public static void main(String[] args) {
        System.out.println("Hello, World!");
    }
}
"""

In [3]:
with open("Hello.java", "w") as fp:
    fp.write(java_source)

Let's store the keyword arguments to `cjdk.java_env()` so that we can call it
several times with the same configuration.

In [4]:
cjdk_config = dict(vendor="graalvm-java17", version="22.1.0")

The GraalVM `native-image` command is not included in the default install, so
we need to use `gu` (the GraalVM updater) to install it.

(On macOS, you may see warnings related to `setrlimit` in this and following
steps. They can be ignored.)

In [5]:
with cjdk.java_env(**cjdk_config):
    subprocess.run(["gu", "install", "native-image"], check=True)

cjdk: Installing JDK graalvm-java17:22.1.0 to /home/runner/.cache/cjdk


Downloading:   0%|          | 0.00/451M [00:00<?, ?B/s]

Extracting: 0files [00:00, ?files/s]

Downloading: Component catalog from www.graalvm.org


Processing Component: Native Image
Downloading: Component native-image: Native Image from github.com
[                    ][#                   ]

[##                  ][###                 ][####                ][#####               ][######              ]

[#######             ][########            ][#########           ][##########          ][###########         ]

[############        ][#############       ][##############      ][###############     ][################    ]

[#################   ][##################  ][################### ][####################]Installing new component: Native Image (org.graalvm.native-image, version 22.1.0)


Now let's compile the source, first with `javac` to byte code, then to a native
image.

In [6]:
with cjdk.java_env(**cjdk_config):
    subprocess.run(["javac", "Hello.java"], check=True)
    subprocess.run(["native-image", "Hello"], check=True)

GraalVM Native Image: Generating 'hello' (executable)...


[1/7] Initializing...                                                                                    (4.7s @ 0.19GB)
 Version info: 'GraalVM 22.1.0 Java 17 CE'
 C compiler: gcc (linux, x86_64, 9.4.0)
 Garbage collector: Serial GC


[2/7] Performing analysis...  [******]                                                                  (21.8s @ 0.83GB)
   2,852 (74.23%) of  3,842 classes reachable
   3,399 (50.83%) of  6,687 fields reachable
  12,932 (44.57%) of 29,017 methods reachable
      27 classes,     0 fields, and   345 methods registered for reflection
      57 classes,    58 fields, and    51 methods registered for JNI access


[3/7] Building universe...                                                                               (1.8s @ 1.03GB)


[4/7] Parsing methods...      [*]                                                                        (1.8s @ 1.29GB)


[5/7] Inlining methods...     [****]                                                                     (3.4s @ 0.88GB)


[6/7] Compiling methods...    [****]                                                                    (20.6s @ 1.65GB)


[7/7] Creating image...                                                                                  (2.0s @ 0.58GB)
   4.41MB (35.36%) for code area:    7,591 compilation units
   6.97MB (55.89%) for image heap:   1,711 classes and 102,362 objects
   1.09MB ( 8.75%) for other data
  12.47MB in total
------------------------------------------------------------------------------------------------------------------------
Top 10 packages in code area:                               Top 10 object types in image heap:
 666.54KB java.util                                         1014.06KB byte[] for general heap data
 338.23KB java.lang                                          984.94KB java.lang.String
 274.64KB java.text                                          944.51KB byte[] for code metadata
 235.70KB java.util.regex                                    645.59KB java.lang.Class
 194.32KB com.oracle.svm.jni                                 572.10KB byte[] for java.lang.String
 193.16KB jav

Finally, let's run the native image.

In [7]:
with cjdk.java_env(**cjdk_config):
    subprocess.run(["./hello"], check=True)

Hello, World!
